# Elektrik Tahsilat ve Tüketim Analizi Görselleştirme Raporu

Bu notebook; **Notebook 1**'de temizlenmiş ve konsolide edilmiş veri setleri üzerinden işletmenin finansal performansını ve operasyonel verimliliğini analiz etmek; Matplotlib ve Seaborn kütüphaneleri kullanılarak elde edilen sonuçları görselleştirmek amacıyla hazırlanmıştır.
Analiz kapsamında;
* **Ödeme Disiplini:** Abonelerin ödeme hızları ve vaktinde ödeme performansları.
* **Müşteri Segment Bazlı Riskler:** Hesap sınıflarına göre (Mesken, Sanayi, Ticari) tahsilat risklerinin dağılımı.
* **Bölgesel Tüketim Trendleri:** Dönemsel ve coğrafi bazda enerji kullanım alışkanlıkları.
* **İlçelerin Tüketim Kapasiteleri:** İlçe bazında ortalama ve toplam kWh tüketim değerleri.
* **Tahsilat Yaşlandırması:** İlçe bazında tahsilat zamanlaması ve toplam tahakkuk içerisindeki ağırlıkları.
* **Müşteri Profilleri:** Segment bazlı abone dağılımları ve tüketim karakteristikleri.  
incelenmektedir.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib.ticker import ScalarFormatter
warnings.filterwarnings("ignore")

In [ ]:
file_path = 'elektrik_verisi_temiz.csv'
file_path_1 = 'df_tahsilat_temiz.xlsx'
file_path_2 = 'df_tahsilat_1_temiz.xlsx'
file_path_3 = 'musteri_risk.csv'

In [ ]:
data = pd.read_csv(file_path)
tahsilat = pd.read_excel(file_path_1)
tahsilat_1 = pd.read_excel(file_path_2)
musteri_risk = pd.read_csv(file_path_3)

In [ ]:
data.info()

## İlçe Bazlı Müşteri Segmentasyonu ve Dağılım Analizi

Bu bölümde, seçili ilçelerdeki (**Hamamözü, Gümüşhacıköy, Göynücek**) abone yapısı ve hesap sınıflarının dağılımı incelenmektedir. 

### Analitik Yaklaşım ve Logaritmik Ölçek Kullanımı
Veri setinde bazı segmentler (örneğin 'Mesken') binlerce aboneye sahipken, diğer segmentler (örneğin 'Sanayi') çok az sayıda aboneden oluşabilmektedir. Bu devasa sayısal farkın aynı grafikte görselleştirilebilmesi ve küçük hacimli ancak kritik segmentlerin görünür kılınması için **Logaritmik Ölçek** tercih edilmiştir.

* **Amaç:** Her ilçenin kendine has abone profilini (Tarım, Mesken, Ticari vb.) sayısal ve yüzdesel olarak ortaya koymak.
* **Görselleştirme Tekniği:** İlçeler arası karşılaştırmayı kolaylaştırmak için `subplots` yapısı kullanılmış; her ilçe için en büyük 10 abone grubu `barplot` ile sıralanmıştır.
* **İstatistiki Çıktı:** Grafiklerin yanı sıra, her ilçe için adet ve yüzde (%) bazında hazırlanan özet tablolar, operasyonel planlama için detaylı veri sunmaktadır.


In [ ]:
#Hesap tiplerine göre dağılım
ilce_listesi = ['HAMAMÖZÜ', 'GÜMÜŞHACIKÖY', 'GÖYNÜCEK']
fig, axes = plt.subplots(1, 3, figsize=(18, 8))
for i, ilce_adi in enumerate(ilce_listesi):
    ilce_verisi = data[data['ilce'] == ilce_adi]
 adet_serisi = ilce_verisi['Hesap Sınıfı'].value_counts()
    yuzde_serisi = ilce_verisi['Hesap Sınıfı'].value_counts(normalize=True) * 100
    
    sonuc = pd.DataFrame({'Adet': adet_serisi, 'Yüzde (%)': yuzde_serisi.round(2)})
    print(f"\n{'='*10} {ilce_adi} İSTATİSTİKLERİ {'='*10}")
    print(sonuc.head(10)) # Grafikteki gibi ilk 10 kategoriyi gösterir
    
    top_classes = adet_serisi.nlargest(10)
    
    sns.barplot(x=top_classes.values,y=top_classes.index,ax=axes[i],hue=top_classes.index,
    palette='viridis',legend=False)
# Logaritmik Ölçek Ayarları
  axes[i].set_xscale('log') 
  axes[i].xaxis.set_major_formatter(ScalarFormatter())
  axes[i].ticklabel_format(style='plain', axis='x')
    
  axes[i].set_title(f'{ilce_adi} Dağılımı\n(Logaritmik Ölçek)', fontsize=14, fontweight='bold')
  axes[i].set_xlabel('Müşteri Sayısı (Log)')
  axes[i].grid(True, axis='x', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.show()

## Mevsimsel Tüketim Trendleri ve İlçe Bazlı Talep Analizi

Bu bölümde, enerji tüketiminin yıl içerisindeki değişimi (mevsimsellik etkisi) ve bu değişimin ilçeler bazındaki farklılıkları incelenmektedir.
Ayrıca ısı grafikleri ve bar grafikleri yardımıyla hangi mevsimde kwH tüketiminin fazla olduğu ve tüketilen ortalama kwH oranları da gösterilmiştir.

### Zaman Serisi ve Trend Analizi
Aylık ortalama tüketim değerleri kullanılarak oluşturulan bu çizgi grafik (Line Plot), işletmeye şu stratejik bilgileri sunmaktadır:

* **Mevsimsellik :** Kış aylarındaki ısınma veya yaz aylarındaki soğutma/tarımsal sulama kaynaklı tüketim artışlarının tespiti.
* **İlçe Karşılaştırması:** Hangi ilçenin enerji talebinin daha stabil olduğu, hangisinin mevsimsel dalgalanmalara daha açık olduğunun belirlenmesi.
* **Operasyonel Planlama:** Tüketimin zirve yaptığı aylarda tahsilat birimlerinin ve teknik ekiplerin iş yükü tahminlemesinin yapılması.

> **Analitik Not:** Grafikteki `estimator='mean'` parametresi, her aydaki tüm abonelerin ortalamasını alarak genel eğilimi görmemizi sağlar. Bu sayede tekil uç değerlerin (outliers) genel trendi saptırması engellenmiştir.

In [ ]:
#%% Mevsimlere göre kwh tüketimi
ay_sirasi = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 
             'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']

plt.figure(figsize=(14, 7))
sns.lineplot(data=data,x='ay',y='kwh',hue='ilce',marker='o', 
estimator='mean',errorbar=None,sort=False)

plt.title('İlçelere Göre Aylık Ortalama Tüketim (kWh)', fontsize=14, fontweight='bold')
plt.xlabel('Aylar', fontsize=12)
plt.ylabel('Ortalama kWh', fontsize=12)
plt.legend(title='İlçeler', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(ticks=range(len(ay_sirasi)), labels=ay_sirasi) #ay sırası sabitleme
plt.tight_layout()
plt.show()

In [ ]:
mevsim_pivot = data.groupby(['ilce', 'Mevsim'])['kwh'].mean().unstack()
plt.figure(figsize=(10, 5))
sns.heatmap(mevsim_pivot, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title('Mevsimsel Tüketim Yoğunluğu (Ortalama kWh)')
plt.show()

In [ ]:
mevsim=data.groupby(['ilce', 'Mevsim'])['kwh'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(data=mevsim, x='Mevsim', y='kwh', hue='ilce', palette='magma')
plt.title('İlçe ve Mevsim Bazlı Ortalama Tüketim (kWh)', fontsize=14, fontweight='bold')
plt.ylabel('Ortalama kWh')
plt.xlabel('Mevsim')
plt.legend(title='İlçe')
plt.show()

## Tahsilat Kanalları ve Bölgesel Dağılım Analizi

Bu bölümde, işletmenin tahsilat yükünün coğrafi ve kurumsal bazda nasıl dağıldığı incelenmektedir. Yapılan görselleştirmeler, operasyonel yoğunluğun hangi noktalarda kümelendiğini ortaya koymaktadır.

### 📊 İncelenen Temel Metrikler:
* **İlçe Bazlı Tahsilat İşlem Dağılımı:** Hangi bölgelerin işlem hacmi açısından daha aktif olduğunun belirlenmesi.
* **En Yoğun 10 Şube/Banka:** Tahsilat işlemlerinde en çok tercih edilen finansal kanalların tespiti.

> **Analiz Notu:** Yüksek işlem hacmine sahip ilçelerde şube sayısının yeterliliği ve bu bölgelerdeki dijital ödeme kanallarının kullanım oranı, operasyonel maliyetleri düşürmek adına kritik önem taşımaktadır.

In [ ]:
#%% İlçe-şube dağılımı ve en yoğun şubeler
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
ilce_adedi = tahsilat['İlçe'].value_counts()
sns.barplot(x=ilce_adedi.index,y=ilce_adedi.values,ax=axes[0], 
hue=ilce_adedi.index,palette='magma',legend=False)

axes[0].set_title('İlçe Bazlı Tahsilat İşlem Dağılımı', fontsize=14, fontweight='bold')
axes[0].set_ylabel('İşlem Adedi')
axes[0].set_xlabel('İlçe')
axes[0].tick_params(axis='x', rotation=45) 

sube_adedi = tahsilat['Şube'].value_counts().nlargest(10)
sns.barplot(x=sube_adedi.values, 
            y=sube_adedi.index, 
            ax=axes[1], 
            hue=sube_adedi.index, 
            palette='viridis', 
            legend=False)

axes[1].set_title('En çok İşlem Yapılan 10 Banka ', fontsize=14, fontweight='bold')
axes[1].set_xlabel('İşlem Adedi')
axes[1].set_ylabel('Şube Adı')
plt.tight_layout()
plt.show()

In [ ]:
islem_sayisi = tahsilat.groupby(['Şube', 'İlçe'])['Söz.hsp.(bağımsız)'].count().reset_index()
islem_sayisi.columns = ['Şube', 'İlçe', 'İşlem_Sayısı'] 
islem_sayisi = islem_sayisi.sort_values('İşlem_Sayısı', ascending=False)


In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=islem_sayisi, x='İşlem_Sayısı', y='İlçe', hue='Şube', dodge=False)
plt.title('Şube ve İlçeye Göre Toplam İşlem Sayısı ', fontsize=14, fontweight='bold')
plt.xlabel('İşlem Adedi (Fatura/Makbuz Sayısı)')
plt.ylabel('İlçe')
plt.legend(title='Şube', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## Genel Ödeme Karakteristiği Dağılımı
Tüm veri setindeki tahsilatların genel karakteristiği incelenmiştir. Bu analiz, nakit akışının ne kadarının sağlıklı (vaktinde) olduğunu, ne kadarının ise riskli (gecikmeli/parçalı) olduğunu göstermektedir.


In [ ]:
#%% Ödeme Oranları
vaktinde_sutunlar = ['Son Ödeme Tarihinden Önceki Tahsilat', 'Son Ödeme Tarihindeki Tahsilat']
gecikmeli_sutunlar = [i for i in tahsilat_1.columns if 'Son Ödeme (' in i]
ozet = tahsilat_1['Ödeme Davranışı'].value_counts()

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(ozet,labels=ozet.index,autopct='%1.1f%%', 
startangle=140,colors=['#2ecc71', '#e74c3c', '#f1c40f', '#95a5a6'], 
explode=[0.05] * len(ozet))
plt.title('Ödeme Karakteristiği Dağılımı', fontsize=14)
plt.show()

In [ ]:
#%% Tahsilat trendi 
tahsilat_1.info()
tahsilat_sutunlari = vaktinde_sutunlar + gecikmeli_sutunlar
trend_ilce = tahsilat_1.groupby(['ay', 'İlçe'])['Tahakkuk Tutar'].sum().unstack().fillna(0)

## İlçe Bazlı Aylık Tahsilat Trendi (Stacked Area Analizi)

Bu bölümde, işletmenin toplam tahsilat hacmi ve bu hacmin ilçeler arasındaki dağılımı zaman serisi bazında incelenmektedir. 

### Yığılmalı Alan Grafiği Analizi
Tahakkuk eden tutarların aylık toplamlarını gösteren bu grafik, finansal performansın sürekliliğini ve bölgesel katkıları anlaşılmasını sağlar.

* **Toplam Hacim Trendi:** Grafiğin en üst çizgisi, işletmenin ilgili aydaki toplam tahsilat kapasitesini temsil eder. Bu sayede mevsimsel bazda nakit girişindeki daralmalar veya artışlar net bir şekilde gözlemlenebilir.
* **Bölgesel Katkı (Share):** Her bir renkli alan, bir ilçenin toplam tahsilat içerisindeki payını gösterir. Alanın kalınlığı, o ilçenin finansal ağırlığını temsil etmektedir.
* **Dinamik Karşılaştırma:** İlçelerin tahsilat trendlerinin birbirine paralel olup olmadığı veya belirli aylarda bir ilçenin diğerlerine göre daha baskın hale gelip gelmediği bu görselleştirme ile analiz edilmektedir.

> **Teknik Not:** Grafikte `stacked=True` parametresi kullanılarak veriler üst üste bindirilmiş, böylece toplam portföy büyüklüğü görselleştirilmiştir. `alpha=0.6` ile sağlanan şeffaflık, katmanlar arasındaki geçişlerin daha yumuşak ve izlenebilir olmasını sağlar.

In [ ]:
plt.figure(figsize=(14, 8))
trend_ilce.plot(kind='area', stacked=True, alpha=0.6, colormap='viridis', figsize=(14, 8))
plt.title('İlçe Bazlı Aylık Toplam Tahsilat Trendi (Stacked Area)', fontsize=15, fontweight='bold')
plt.xlabel('Aylar', fontsize=12)
plt.ylabel('Toplam Tahsilat Tutarı (TL)', fontsize=12)
plt.legend(title='İlçeler', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.xticks(trend_ilce.index)
plt.tight_layout()
plt.show()

In [ ]:
#%% İlçeler arası ödeme hızı
tahsilat_1.info()
odeme_hizi = tahsilat_1.groupby(['İlçe','Ödeme Davranışı']).size().unstack(fill_value=0)
odeme_hizi_oran = odeme_hizi.div(odeme_hizi.sum(axis=1), axis=0) * 100
odeme_hizi_oran = odeme_hizi_oran.reset_index().melt(id_vars='İlçe', var_name='Ödeme Tipi', value_name='Oran')

In [ ]:
plt.figure(figsize=(14, 7))
ax = sns.barplot(data=odeme_hizi_oran, x='İlçe', y='Oran', hue='Ödeme Tipi', palette='viridis')

plt.title('İlçeler Arası Ödeme Hızı ve Durumları (%)', fontsize=15)
plt.ylabel('Ödeme Oranı (%)')
plt.xlabel('İlçe')
plt.ylim(0, 100) 
plt.legend(title='Ödeme Durumu', bbox_to_anchor=(1.05, 1), loc='upper left')

for p in ax.patches:
    if p.get_height() > 0: 
        ax.annotate(f'%{p.get_height():.1f}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', fontsize=10, color='black', 
                    xytext=(0, 7), textcoords='offset points', fontweight='bold')

plt.tight_layout()
plt.show()

## İlçe Bazında Tüketim (kWh) Dağılımı
İlçeler arasındaki enerji tüketim alışkanlıklarını ve uç değerleri (sanayi tipi yüksek tüketimler) gözlemlemek için kutu grafiği (Boxplot) kullanılmıştır.Her ilçe için oluşturulan kutu grafikleri ile medyan kwH değerleri ve aykırı değerler daha kolay gözlemlenebilir hale getirilmiştir.

* **İyileştirme:** Verideki aşırı yüksek tüketimli abonelerin kutu görünürlüğünü bozmaması için aykırı değerler yönetilerek grafik netleştirilmiştir.

In [ ]:
#%% Aykırı Değer Boxplot
q1 = data['kwh'].quantile(0.25)
q3 = data['kwh'].quantile(0.75)
iqr = q3 - q1
ust_sinir = q3 + 1.5 * iqr

In [ ]:
# İlçelere göre kWh dağılımı
plt.figure(figsize=(12, 6))
sns.boxplot(data=data, x='ilce', y='kwh', palette='Set2')
plt.title('İlçe Bazında kWh Tüketim Dağılımı ', fontsize=14)
plt.xlabel('İlçe')
plt.ylabel('kWh Tüketimi')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Medyan kwh tüketime göre sıralama
plt.figure(figsize=(14, 7))
order = data.groupby('ilce')['kwh'].median().sort_values(ascending=False).index
sns.boxplot(data=data, x='ilce', y='kwh', order=order,palette='viridis', showfliers=False) 
plt.title('İlçe Bazında kWh Tüketim Dağılımı (Genel Eğilim)', fontsize=15, fontweight='bold')
plt.xlabel('İlçe')
plt.ylabel('kWh Tüketimi (Aykırı Değerler Hariç)')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.xticks(rotation=45)
plt.show()

In [ ]:
data['fatura_tarihi'] = pd.to_datetime(data['fatura_tarihi'])
data['kayit_tarihi']  = pd.to_datetime(data['kayit_tarihi'])
data['vade_tarihi']   = pd.to_datetime(data['vade_tarihi'])
data['mali_yil_donem'] = pd.to_datetime(data['mali_yil_donem'])
data['yil'] = data['mali_yil_donem'].dt.year

In [ ]:
# Yıl bazlı kwH Tüketimi
mevsim_yillik = data.groupby(['yil', 'Mevsim'])['kwh'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(data=mevsim_yillik,x='Mevsim', y='kwh', hue='yil', palette='viridis')
plt.title('Yıllara Göre Mevsimsel Tüketim Karşılaştırması', fontsize=14, fontweight='bold')
plt.ylabel('Ortalama kWh')
plt.legend(title='Yıl')
plt.show()

In [ ]:
#Aylık tüketim yıllık karşılaştırma
plt.figure(figsize=(14, 7))
sns.lineplot(data=data, x='ay', y='kwh', hue='yil', marker='o', estimator='mean', errorbar=None)
plt.title('Aylık Tüketim Trendi: Yıllık Karşılaştırma', fontsize=14, fontweight='bold')
plt.xlabel('Aylar')
plt.ylabel('Ortalama kWh')
plt.grid(True, alpha=0.3)
plt.show()

##  Hesap Sınıfı Bazlı Enerji Tüketim Analizi (kWh)

Bu bölümde, toplam enerji tüketiminin müşteri segmentleri (Hesap Sınıfları) arasındaki dağılımı incelenmektedir. Bu analiz, işletmenin gelir modelini ve enerji yükü dağılımını anlamak için kritiktir.

* **Amaç:** Enerji yoğunluğunun hangi kullanım amaçlarında (Mesken, Sanayi, Ticarethane vb.) toplandığını belirlemek.
* **Kritik Gösterge:** Bazı segmentler sayıca (adet) çok fazla olmasına rağmen toplam tüketimde (kWh) küçük bir pay alırken; Sanayi gibi segmentler az sayıda abone ile devasa tüketim gerçekleştirebilir.
* **Analitik Yaklaşım:** Aykırı değerlerin (uç tüketimler) tespiti ve bu değerlerin toplam tahsilat riski ile korelasyonu bu aşamada değerlendirilmektedir.

> **İpucu:** Yüksek tüketim yapan "Hesap Sınıfları"ndaki küçük bir gecikme oranı, düşük tüketimli gruplardaki büyük gecikme oranlarından daha fazla finansal risk yaratabilir.

In [ ]:
#%% hesap sınıfı-ilçeye göre kwH tüketimi
ilce_hesap_siniflari = data.groupby(['ilce', 'Hesap Sınıfı'])['kwh'].mean().unstack().fillna(0).round(2)
plt.figure(figsize=(15, 8))
sns.heatmap(ilce_hesap_siniflari,annot=True, fmt=".1f", cmap='coolwarm', linewidths=0.5)
plt.title('İlçe ve Hesap Sınıfı Bazlı Ortalama kwH Tüketim Hacmi', fontsize=15, fontweight='bold', pad=20)
plt.xlabel('Hesap Sınıfı (Abone Grubu)', fontsize=12)
plt.ylabel('İlçe', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#Logaritmik ölçekli
heatmap_log_data = data.groupby(['ilce', 'Hesap Sınıfı'])['kwh'].apply(lambda x: np.log1p(x).mean()).unstack().fillna(0)
plt.figure(figsize=(16, 9))
sns.heatmap(heatmap_log_data, annot=True, fmt=".1f", cmap="YlOrRd", linewidths=0.5)
plt.title('İlçe ve Hesap Sınıfı Bazlı Ortalama Tüketim Yoğunluğu (Logaritmik Ölçek)', fontsize=15, fontweight='bold', pad=20)
plt.xlabel('Hesap Sınıfı (Abone Grubu)', fontsize=12)
plt.ylabel('İlçe', fontsize=12)
plt.tight_layout()
plt.show()

## Müşteri Gruplarına Göre Risk Analizi
Bu bölümde, "Hangi müşteri grubu daha geç ödeme yapıyor?" sorusuna yanıt aranmaktadır. 

* **Filtreleme:** İstatistiksel sapmaları önlemek amacıyla, toplam işlem hacmi 1000 TL'nin altında kalan düşük hacimli segmentler "Anlamlı Risk" analizi kapsamında elenmiştir.
* **Kritik Eşik:** Grafikteki mavi kesikli çizgi (%13.8), işletmenin genel gecikme ortalamasını temsil etmektedir.

In [ ]:
#%% Müşteri Grubu Risk Grafiği
plt.figure(figsize=(12, 8))
ax = sns.barplot(data=musteri_risk.reset_index(), x='Risk_Orani', y='Hesap Sınıfı', palette='Reds_r')
sns.barplot(data=musteri_risk, x='Risk_Orani', y='Hesap Sınıfı', palette='Reds_r')
plt.title('Hangi Müşteri Grubunun Geç Ödeme Riski Yüksek? ', fontsize=14)
plt.xlabel('Gecikmeli Ödeme Yüzdesi (%)')
plt.axvline(13.8, color='blue', linestyle='--', label='Genel Ortalama (%13.8)') 
#oranlar sütun sonları yazar
for p in ax.patches:
    ax.annotate(f'%{p.get_width():.1f}', 
                (p.get_width(), p.get_y() + p.get_height() / 2), 
                ha='left', va='center', xytext=(5, 0), 
                textcoords='offset points')
plt.legend()
plt.show()

## 3. İlçe Bazlı Tahsilat Zamanlamaları (%)

Notebook 1'de oluşturulan **Master Veri Seti** kullanılarak ilçe bazında tahsilat zamanlamaları üç kategoride incelenerek sınıflandırılmış, toplam tahakkuk içerisindeki paylarına da yer verilmiştir.

* **Sınıflandırma:** Tahsilatlar Master veri setinden filtreleme yolu ile şu şekilde gruplandırılmıştır:
    * **Vaktinde Ödenen:** Son ödeme tarihine kadar kasaya giren nakit akışını,
    * **Kısa Vadeli Gecikme (1-10 Gün):** Genellikle unutkanlık veya kısa süreli nakit sıkışıklığını,
    * **Uzun Vadeli Gecikme (60+ Gün):** Şüpheli alacak potansiyeli taşıyan ve hukuki takip gerektirecek kritik risk grubudur.
* **Yüzde Oranı Hesaplama:** Her bir kategorinin ilçe bazlı toplam tahakkuk içerisindeki ağırlığını görmek adına yüzdelik oranları hesaplanmıştır.
* **Analiz Amacı:** İlçeler arasındaki ödeme disiplini farklarını saptayarak, saha operasyonlarının (kesme-bağlama/icra) hangi bölgelerde yoğunlaşması gerektiğini belirlemektir.

In [ ]:
#%% İlçe bazında tahsilatlar
file_path_4 ="ilce_tahsilat.csv"
df_master = pd.read_csv(file_path_4)

grafik_data = df_master.groupby('İlçe').agg({
    'Vaktinde_Odenen': 'sum',
    'Kisa_Vadeli_Gecikme': 'sum',
    'Uzun_Vadeli_Gecikme': 'sum'
}).reset_index()

toplam = grafik_data[['Vaktinde_Odenen', 'Kisa_Vadeli_Gecikme', 'Uzun_Vadeli_Gecikme']].sum(axis=1)
grafik_data['Vaktinde_Oran'] = (grafik_data['Vaktinde_Odenen'] / toplam) * 100
grafik_data['Kisa_Oran'] = (grafik_data['Kisa_Vadeli_Gecikme'] / toplam) * 100
grafik_data['Uzun_Oran'] = (grafik_data['Uzun_Vadeli_Gecikme'] / toplam) * 100

plt.figure(figsize=(12, 8))
plt.bar(grafik_data['İlçe'], grafik_data['Vaktinde_Oran'], label='Vaktinde', color='#00a8a8')
plt.bar(grafik_data['İlçe'], grafik_data['Kisa_Oran'], bottom=grafik_data['Vaktinde_Oran'], 
        label='Kısa Vadeli Gecikme (1-10 Gün)', color='#58508d')
plt.bar(grafik_data['İlçe'], grafik_data['Uzun_Oran'], bottom=grafik_data['Vaktinde_Oran'] + grafik_data['Kisa_Oran'], 
        label='Uzun Vadeli Gecikme (60+ Gün)', color='#bb3e03')

plt.title('İlçe Bazında Tahsilat Zamanlaması (%)', fontsize=15, fontweight='bold')
plt.ylabel('Toplam Tahakkuk İçindeki Payı (%)')
plt.xlabel('İlçeler')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()